In [254]:
import pandas as pd
import requests
import json

from keys import *

In [114]:
# !pip install --upgrade azure-cognitiveservices-language-textanalytics

In [116]:
# from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
# from msrest.authentication import CognitiveServicesCredentials

In [256]:
!pip install nltk

Traceback (most recent call last):
  File "/data/anaconda/envs/py35/bin/pip", line 7, in <module>
    from pip import main
ImportError: cannot import name 'main'


In [221]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

ModuleNotFoundError: No module named 'nltk'

In [250]:
sample_tweets = pd.DataFrame.from_csv(
    "sample_tweets.csv",
    index_col=None
)

In [169]:
sample_tweets["id"] = sample_tweets.index
sample_tweets.rename(columns={"Body text": "text"}, inplace=True)
sample_tweets.text = sample_tweets.text.str.lower()
sample_tweets.head(2)

,Down ID,Posted time,text,Point latitiude,Point longitude,Bio location,Image url,id
0,"""tag:search.twitter.com,2005:1109081183105835008""",22/03/2019 13:15,"""primeape was recently spotted trying to make ...",NaN,NaN,NaN,"""https://pbs.twimg.com/profile_images/76017773...",0
1,"""tag:search.twitter.com,2005:1109080961277407234""",22/03/2019 13:14,"""@gianninewbon meet me at london bridge at hal...",NaN,NaN,"""Derbados""","""https://pbs.twimg.com/profile_images/10569615...",1


In [144]:
sample_tweets.count()

Down ID            44816
Posted time        44816
text               44816
Point latitiude      701
Point longitude      701
Bio location       31946
Image url          42978
id                 44816
dtype: int64

In [195]:
documents_str = sample_tweets[["id", "text"]].head(1000).to_json(orient = "records")
documents_json = json.loads(json.dumps({"documents" : json.loads(documents_str)}))

In [196]:
len(documents_json["documents"])

1000

In [255]:
keyphrase_url = text_analytics_base_url + "keyPhrases"
entities_url = text_analytics_base_url + "entities"

In [183]:
headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(keyphrase_url, headers=headers, json=documents_json)
key_phrases = response.json()

In [184]:
key_phrases_df = pd.DataFrame.from_dict(key_phrases['documents'])
key_phrases_df.head()

,id,keyPhrases
0,0,"[friends, tower bridge, primeape]"
1,1,"[gianninewbon, london bridge]"
2,2,"[brown fog, winter dawn, crowd, london bridge,..."
3,3,"[london eye, iconic london open, double decker..."
4,4,"[taproom, londoncycling, cyclemeets, londonbar..."


In [175]:
(key_phrases_df["keyPhrases"].apply(lambda x: len(x)).mean(),
key_phrases_df["keyPhrases"].apply(lambda x: len(x)).max(),
key_phrases_df["keyPhrases"].apply(lambda x: len(x)).min())

(5.81, 32, 1)

In [224]:
unique_key_phrases = set(sum(key_phrases_df["keyPhrases"].values, []))

In [227]:
key_phrases_category = {}
for key in unique_key_phrases:
    key_phrases_category[key] = []

In [238]:
def map_categories(row):
    for phrase in row["keyPhrases"]:
        key_phrases_category[phrase].append(row["id"])
    return None

In [247]:
key_phrases_df.apply(map_categories, axis=1)

In [246]:
key_phrases_category_count={}
for key, value in key_phrases_category.items():
    key_phrases_category_count[key] = len(value)

In [248]:
from collections import OrderedDict
key_phrases_category_count = OrderedDict(
                               sorted(key_phrases_category_count.items(), 
                                      key=lambda kv: kv[1], 
                                      reverse=True)
                            )

In [249]:
key_phrases_category_count

OrderedDict([('london bridge', 810),
             ('london bridge terrorist attack', 238),
             ('days', 238),
             ('interior department spokeswoman', 236),
             ('islamophobic', 236),
             ('podcast episode', 222),
             ('immigrant rhetoric', 210),
             ('tower bridge', 202),
             ('huffpostpol', 200),
             ('eb', 64),
             ('west sacramento', 64),
             ('time', 62),
             ('london', 60),
             ('people', 54),
             ('minutes', 48),
             ('london bridge station', 46),
             ('se', 40),
             ('tower bridge gtwy', 38),
             ('railway', 36),
             ('fair lady', 34),
             ('day', 34),
             ('battersea', 30),
             ('morning', 30),
             ('heavy traffic jam', 30),
             ('train', 28),
             ('inconvenience', 28),
             ('london bridge attack', 28),
             ('trains', 26),
             ('harbor blv

In [170]:
response  = requests.post(entities_url, headers=headers, json=documents_json)
entities = response.json()

In [198]:
entities_df = pd.DataFrame.from_dict(entities['documents'])
entities_df.head()

,entities,id
0,"[{'name': 'Tower Bridge', 'matches': [{'text':...",0
1,"[{'name': 'London Bridge', 'matches': [{'text'...",1
2,"[{'name': 'London Bridge', 'matches': [{'text'...",2
3,"[{'name': 'Buckingham Palace', 'matches': [{'t...",3
4,"[{'name': 'London Bridge', 'matches': [{'text'...",4


In [177]:
(entities_df["entities"].apply(lambda x: len(x)).mean(),
entities_df["entities"].apply(lambda x: len(x)).max(),
entities_df["entities"].apply(lambda x: len(x)).min())

(3.628, 18, 0)